In [2]:
probs_seq1 = [[
            0.06390443, 0.42124858, 0.27323887, 0.06870235, 0.0361254,
            0.18184413, 0.16493624
        ], [
            0.03309247, 0.42866108, 0.24390638, 0.09699597, 0.31895462,
            0.0094893, 0.06890021
        ], [
            0.218104, 0.49992557, 0.18245131, 0.08503348, 0.14903535,
            0.08424043, 0.08120984
        ], [
            0.12094152, 0.41162472, 0.01473646, 0.28045061, 0.24246305,
            0.05206269, 0.09772094
        ], [
            0.9333387, 0.40550838, 0.00301669, 0.21745861, 0.20803985,
            0.41317442, 0.01946335
        ], [
            0.16468227, 0.4180699, 0.1906545, 0.53963251, 0.19860937,
            0.04377724, 0.01457421
        ]]

probs_seq2 = [[
            1.06390443, 0.42124858, 0.27323887, 0.06870235, 0.0361254,
            0.18184413, 0.16493624
        ], [
            0.03309247, 0.42866108, 0.24390638, 0.09699597, 0.31895462,
            0.0094893, 0.06890021
        ], [
            0.218104, 0.49992557, 0.18245131, 0.08503348, 0.14903535,
            0.08424043, 0.08120984
        ], [
            0.12094152, 0.41162472, 0.01473646, 0.28045061, 0.24246305,
            0.05206269, 0.09772094
        ], [
            0.9333387, 0.40550838, 0.00301669, 0.21745861, 0.20803985,
            0.41317442, 0.01946335
        ], [
            0.16468227, 0.4180699, 0.1906545, 0.53963251, 0.19860937,
            0.04377724, 0.01457421
        ]]

In [7]:
def find_path(prob, path):
    import ctc
    import time
    import matplotlib.pyplot as plt

    import torch
    import torch.nn.functional as F
    torch.set_printoptions(precision=2)


    # import ctc

    T, B, C = len(prob), 1, len(prob[0])
    t = len(path)
    blank = 0
    device = 'cuda'
    seed = 1
    atol = 1e-3
#     for set_seed in [torch.manual_seed] + ([torch.cuda.manual_seed_all] if device == 'cuda' else []):
#         set_seed(seed)
    tictoc = lambda: (device == 'cuda' and torch.cuda.synchronize()) or time.time()

    # logits = torch.randn(T, B, C, device = device).requires_grad_()
    import numpy
    logits = torch.reshape(torch.as_tensor(numpy.array(numpy.log(prob)),device = device), (T,B,C)).requires_grad_()

    # targets = torch.randint(blank + 1, C, (B, t), dtype = torch.long, device = device)
    targets = torch.reshape(torch.as_tensor(path, dtype = torch.long, device = device), (B, t))



    input_lengths = torch.full((B,), T, dtype = torch.long, device = device)
    target_lengths = torch.full((B,), t, dtype = torch.long, device = device)
    log_probs = logits.log_softmax(dim = -1)
    print(log_probs)
    if print_flag:
        print("log_probs",log_probs)
    if print_flag:
        print('Device:', device)
    if print_flag:
        print('Log-probs shape (time X batch X channels):', 'x'.join(map(str, log_probs.shape)))

    tic = tictoc()
    builtin_ctc = F.ctc_loss(log_probs, targets, input_lengths, target_lengths, blank = 0, reduction = 'none')
    print(builtin_ctc)
    toc = tictoc()
    builtin_ctc_grad, = torch.autograd.grad(builtin_ctc.sum(), logits, retain_graph = True)
    print('Built-in CTC loss', 'fwd', toc - tic, 'bwd', tictoc() - toc)

#     tic = tictoc()
#     custom_ctc = ctc_loss(log_probs, targets, input_lengths, target_lengths, blank = 0, reduction = 'none')
#     toc = tictoc()
#     custom_ctc_grad, = torch.autograd.grad(custom_ctc.sum(), logits, retain_graph = True)
#     print('Custom CTC loss', 'fwd', toc - tic, 'bwd', tictoc() - toc)

#     ce_alignment_targets = ctc_alignment_targets(log_probs, targets, input_lengths, target_lengths, blank = 0)
#     ce_ctc = -ce_alignment_targets * log_probs
#     ce_ctc_grad, = torch.autograd.grad(ce_ctc.sum(), logits, retain_graph = True)

#     print('Custom loss matches:', torch.allclose(builtin_ctc, custom_ctc, atol = atol))
#     print('Grad matches:', torch.allclose(builtin_ctc_grad, custom_ctc_grad, atol = atol))
#     print('CE grad matches:', torch.allclose(builtin_ctc_grad, ce_ctc_grad, atol = atol))

#     alignment = ctc_loss(log_probs, targets, input_lengths, target_lengths, blank = 0, reduction = 'none', alignment = True)
#     a = alignment[:, 0, :target_lengths[0]]
#     # print(a.t().cpu())
#     plt.subplot(311)
#     plt.title('Input-Output maxt CTC Viterbi alignment')
#     plt.imshow(a.t().cpu(), origin = 'lower', aspect = 'auto')
#     plt.xlabel('Input steps')
#     plt.ylabel('Output steps')
#     plt.subplot(312)
#     plt.title('CTC alignment targets')
#     a = ce_alignment_targets[:, 0, :]
#     plt.imshow(a.t().cpu(), origin = 'lower', aspect = 'auto')
#     plt.xlabel('Input steps')
#     plt.ylabel(f'Output symbols, blank {blank}')
#     plt.subplots_adjust(hspace = 0.5)
#     plt.savefig('alignment.png')
#     torch.set_printoptions(precision=2)
#     # print(a.t().cpu())
#     # print(targets[:,0])
#     # print(logits)

#     plt.subplot(313)
#     alignment = ctc_max_loss(log_probs, targets, input_lengths, target_lengths, blank = 0, reduction = 'none', alignment = True)
#     a = alignment[:, 0, :target_lengths[0]]
#     # print(a.t().cpu())
#     plt.title('Input-Output max prob. Viterbi alignment')
#     plt.imshow(a.t().cpu(), origin = 'lower', aspect = 'auto')
#     plt.xlabel('Input steps')
#     plt.ylabel('Output steps')

In [10]:
print_flag = False
find_path(probs_seq2, [1,2,3])

tensor([[[-0.73, -1.66, -2.09, -3.47, -4.11, -2.50, -2.60]],

        [[-3.59, -1.03, -1.59, -2.52, -1.33, -4.84, -2.86]],

        [[-1.79, -0.96, -1.96, -2.73, -2.17, -2.74, -2.77]],

        [[-2.31, -1.09, -4.42, -1.47, -1.62, -3.15, -2.52]],

        [[-0.86, -1.69, -6.59, -2.31, -2.36, -1.67, -4.73]],

        [[-2.25, -1.32, -2.11, -1.07, -2.07, -3.58, -4.68]]], device='cuda:0',
       dtype=torch.float64, grad_fn=<LogSoftmaxBackward>)
tensor([6.07], device='cuda:0', dtype=torch.float64, grad_fn=<CtcLossBackward>)
Built-in CTC loss fwd 0.0010409355163574219 bwd 0.00039005279541015625


In [22]:
# torch.manual_seed(66)
torch.randint(blank + 1, C, (B, t), dtype = torch.long, device = device)


tensor([[5, 6, 3]], device='cuda:0')